In [ ]:
# Libraries
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
# Data loading
data=pd.read_excel("")
print(data)

                       INDIAN_IMPORTER_NAME  ...    Product_ID
0               RADIANT CORPORATION PVT LTD  ...     ['NP101']
1               RADIANT CORPORATION PVT LTD  ...     ['NP101']
2      AXALTA COATING SYSTEMS INDIA PVT LTD  ...    ['MIX181']
3      AXALTA COATING SYSTEMS INDIA PVT LTD  ...    ['MIX181']
4      AXALTA COATING SYSTEMS INDIA PVT LTD  ...    ['MIX181']
...                                     ...  ...           ...
11689  SOUTHERN ROCKS AND MINERALS PVT LTD.  ...            []
11690  SOUTHERN ROCKS AND MINERALS PVT LTD.  ...            []
11691  SOUTHERN ROCKS AND MINERALS PVT LTD.  ...            []
11692                   RADIAN PRODUCTS LLP  ...    ['SH2217']
11693                     AXA LEATHER GROUP  ...  ['E02694TI']

[11694 rows x 3 columns]


In [ ]:
#data["PRODUCT_DESCRIPTION"] = data["PRODUCT_DESCRIPTION"].str.replace('[\d{1,2}.\d{1,2}.\d{2,4}]',"")

In [ ]:
data["PRODUCT_DESCRIPTION"] = data["PRODUCT_DESCRIPTION"].str.replace('\(|\)'," ")

In [ ]:
data["PRODUCT_DESCRIPTION"] = data["PRODUCT_DESCRIPTION"].str.replace('[^a-zA-Z\d\s+]',"")

In [ ]:
data["PRODUCT_DESCRIPTION"] = data["PRODUCT_DESCRIPTION"].apply(lambda x: re.sub('\d\d\d\d\d+', '',x))

In [ ]:
data["PRODUCT_DESCRIPTION"]

0                                NP101 HF  CC DBL DARKGREY
1                                   NP101 HF  CC DBL WHITE
2         PAINT RELATED MATERIAL    STANDOBLUE BASECOAT...
3         PAINT RELATED MATERIAL    STANDOBLUE BASECOAT...
4         PAINT RELATED MATERIAL    STANDOBLUE BASECOAT...
                               ...                        
11689             COLOR ADDITIVESQUARTZ SLABS  IRON BROWN 
11690             COLOR ADDITIVESQUARTZ SLABS  IRON BLACK 
11691            COLOR ADDITIVESQUARTZ SLABS  ULTRAMARINE 
11692    BTD11S2  LOT SH2217  TITANIUM DIOXIDE  TRIETHO...
11693                            SOLEDA IRIDE BIANCON  ETI
Name: PRODUCT_DESCRIPTION, Length: 11694, dtype: object

In [ ]:
data['Product_ID'] = data.Product_ID.astype(str).str.replace('\[|\]|\'', '')

In [ ]:
data["Product_ID"]

0           NP101
1           NP101
2          MIX181
3          MIX181
4          MIX181
           ...   
11689            
11690            
11691            
11692      SH2217
11693    E02694TI
Name: Product_ID, Length: 11694, dtype: object

In [ ]:
data.head()

,PRODUCT_DESCRIPTION,Product_ID
0,15208 NP101HF CC DBL DARKGREY,NP101
1,15209 NP101HF CC DBL WHITE,NP101
2,PAINT RELATED MATERIAL 4024669500813 02050081 ...,MIX181
3,PAINT RELATED MATERIAL 4024669500813 02050081 ...,MIX181
4,PAINT RELATED MATERIAL 4024669500813 02050081 ...,MIX181


In [ ]:
fe = TfidfVectorizer(
    min_df=1,
    max_df=1.0,
    ngram_range=(1, 4),
    max_features=5000,
    stop_words=ENGLISH_STOP_WORDS,
    decode_error='ignore'
    
)

In [ ]:
x_data = fe.fit_transform(data["PRODUCT_DESCRIPTION"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data, data['Product_ID'], test_size=0.2, random_state=1)

In [ ]:
X_train.shape, X_test.shape,y_train.shape, y_test.shape

((9355, 5000), (2339, 5000), (9355,), (2339,))

In [ ]:
"""logistic = linear_model.LogisticRegression()
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)
hyperparameters = dict(C=C, penalty=penalty)
clf1 = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)


best_model = clf1.fit(X_train, y_train)
best_model.best_estimator_.get_params()"""

In [ ]:
model = OneVsRestClassifier(estimator=LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False))

In [ ]:
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


OneVsRestClassifier(estimator=LogisticRegression(C=59.94842503189409,
                                                 class_weight=None, dual=False,
                                                 fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [ ]:
predicted = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

                   0.83      0.95      0.88       466
         018       1.00      1.00      1.00         1
         020       0.00      0.00      0.00         0
     0342LMB       1.00      1.00      1.00         1
      0826HP       0.00      0.00      0.00         1
         100       1.00      1.00      1.00         2
       10000       0.00      0.00      0.00         1
        100C       0.00      0.00      0.00         0
        100F       0.00      0.00      0.00         2
      100MPM       1.00      0.67      0.80         3
        100S       0.00      0.00      0.00         1
         103       1.00      1.00      1.00         1
        1109       0.00      0.00      0.00         1
         113       1.00      1.00      1.00         8
        1171       1.00      1.00      1.00         5
         119       1.00      1.00      1.00         1
         120       1.00      1.00      1.00         1
         121       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
print(confusion_matrix(y_test, predicted))

[[443   0   0 ...   0   0   1]
 [  0   1   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...  35   0   0]
 [  0   0   0 ...   0   5   0]
 [  0   0   0 ...   0   0   0]]


In [ ]:
test_data=pd.read_excel(".xlsx")

In [ ]:
test_data["PRODUCT_DESCRIPTION"] = test_data["PRODUCT_DESCRIPTION"].str.replace('[^a-zA-Z\d\s+]',"")

In [ ]:
test_data['Product_ID'] = test_data.Product_ID.astype(str).str.replace('\[|\]|\'', '')

In [ ]:
test_data["Product_ID"]

0      LR972
1      LR972
2      LR972
3      LR972
4      LR972
       ...  
741         
742         
743         
744         
745         
Name: Product_ID, Length: 746, dtype: object

In [ ]:
x_test_data = fe.transform(test_data["PRODUCT_DESCRIPTION"])

In [ ]:
predictedtest = model.predict(x_test_data)

In [ ]:
test_data['predicted'] = predictedtest

In [ ]:
test_data.to_csv('')